In [4]:
import pandas as pd
import numpy as np
from collections import Counter
from sklearn.model_selection import train_test_split

def XyData(csv_path):
    usecols = ['content', 'type']
    df = pd.read_csv(csv_path, usecols=usecols)

    #['unreliable', 'fake', 'clickbait', 'conspiracy', 'reliable', 'bias', 'hate', 'junksci', 'political', nan, 'unknown']
    df.dropna(subset=['type'], inplace = True)
    #['unreliable', 'fake', 'clickbait', 'conspiracy', 'reliable', 'bias', 'hate', 'junksci', 'political', 'unknown']
    df.drop(df[df['type'] == 'unknown'].index, inplace=True)
    #['unreliable', 'fake', 'clickbait', 'conspiracy', 'reliable', 'bias', 'hate', 'junksci', 'political']
    # DifferentTypes = Counter()
    # DifferentTypes.update(df['type'].to_list())
    # print(DifferentTypes.keys())
    # print(DifferentTypes['fake'])
    df.reset_index(drop=True, inplace=True) #reset index

    y = pd.DataFrame({'result' : np.where(df['type'] == 'fake', 1, 0)}) #[1, 0] = [fake, not fake]
    # fakeVsNonfake = Counter()
    # fakeVsNonfake.update(y)
    # print(fakeVsNonfake)

    unique_words = Counter()
    df['content'].str.split().apply(unique_words.update)
    unique_words.subtract(unique_words) #keep keys, reset count
    # print(unique_words.keys())
    # print(unique_words['sometim'])
    # print(len(unique_words.keys()))
 
    frames = []  
    for row in range(len(df)):
        words = unique_words.copy()
        words.update(df.loc[row,"content"].split())
        frames += [pd.DataFrame(words, index=[row])]
    
    X = pd.concat(frames)
         
    return X, y

# X, y = XyData('..\\finished.csv')

# X.to_csv('X.csv', index=False)
# y.to_csv('y.csv', index=False)

def Split(X_path, y_path):
    # usecols = ['content', 'type']
    # df = pd.read_csv(csv_path, usecols=usecols)
    
    df_X = df = pd.read_csv(X_path)
    df_y = df = pd.read_csv(y_path)
    X_train, X_test, y_train, y_test = train_test_split(df_X,
                                                        df_y,
                                                        train_size = 0.8,
                                                        random_state = 0,
                                                        shuffle = False,
                                                        stratify = None)

    X_test, X_val, y_test, y_val = train_test_split(X_test,
                                                    y_test,
                                                    test_size= 0.5,
                                                    random_state = 0,
                                                    shuffle = False,
                                                    stratify = None)

    return X_train, X_test, X_val, y_train, y_test, y_val
X_train, X_test, X_val, y_train, y_test, y_val = Split('X.csv', 'y.csv')

In [5]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score

model = LinearRegression()
fit = model.fit(X_train, y_train['result'])
y_pred = fit.predict(X_test)
y_pred = [round(x,0) for x in y_pred]

mse = mean_squared_error(y_test, y_pred)
print("LinearRegression MSE: ", mse)
acc = accuracy_score(y_test, y_pred)
print("LinearRegression accuracy: ", acc)

LinearRegression MSE:  0.391304347826087
LinearRegression accuracy:  0.6086956521739131


In [6]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score

model = LogisticRegression(random_state=0, max_iter=1000)
fit = model.fit(X_train, y_train['result'])
y_pred = fit.predict(X_test)
y_pred = [round(x,0) for x in y_pred]

mse = mean_squared_error(y_test, y_pred)
print("LinearRegression MSE: ", mse)
acc = accuracy_score(y_test, y_pred)
print("LinearRegression accuracy: ", acc)

LinearRegression MSE:  0.17391304347826086
LinearRegression accuracy:  0.8260869565217391
